In [ ]:
''' this is a script designed to take in a dump file from an MD LAMMPS simulation and process it using OVITO.
it will export the first frame without modifiers (original crystal), the last frame with modifiers (interstitials),
and a pared down file containing interstitials throughout the course of the simulation in order to analyze recombination effects.
@author:wuaudrey'''

In [ ]:
!pip install ovito
import ovito
from ovito import scene
from ovito.io import FileWriterInterface, import_file, export_file
from ovito.modifiers import ComputePropertyModifier, ExpressionSelectionModifier, CoordinationAnalysisModifier, DeleteSelectedModifier, SliceModifier
import sys

In [ ]:
#pipeline = import_file('Si_quartz_atomic_1keV_28282_5ps_2025_07_07.dump')

#if running with bash file, comment out the line above and use:
file_path = sys.argv[1]
pipeline = import_file(file_path)

# insert the pipeline into the visualization scene.
pipeline.add_to_scene()
# it's now part of the 'scene.pipelines' list.
assert pipeline in scene.pipelines
# if needed, we can take it out again.
#pipeline.remove_from_scene()

# lattice dimensions
xdim, ydim, zdim = 200.0, 200.0, 200.0

In [ ]:
# add modifiers

# compute property modifier
pipeline.modifiers.append(ComputePropertyModifier(
    output_property = 'Timestep',
    expressions = ['Timestep']
))

# coordination analysis modifier
pipeline.modifiers.append(CoordinationAnalysisModifier(cutoff = 2.0, number_of_bins = 200))

In [ ]:
data_original = pipeline.compute()

# first frame:
export_file(data_original, "quartz_600k_atomic_1keV_28282_og_python.xyz", "xyz", frame = 0, multiple_frames = False, columns =
  ["Particle Identifier", "Particle Type", "Position.X", "Position.Y", "Position.Z", "Timestep", "Coordination"])

In [ ]:
# add modifiers pt 2

# expression selection modifier
pipeline.modifiers.append(ExpressionSelectionModifier(expression = 'ParticleType == 1 && Coordination == 4 || ParticleType == 2 && Coordination == 2'))

# slice modifiers
'''distance = 0.0 only for lattices centered at the origin, if your lattice isn't centered at the origin, you need to change the distance value'''

pipeline.modifiers.append(SliceModifier(distance=0.0, slab_width = xdim - 10, normal = (1.0,0.0,0.0)))
pipeline.modifiers.append(SliceModifier(distance=0.0, slab_width = ydim - 10, normal = (0.0,1.0,0.0)))
pipeline.modifiers.append(SliceModifier(distance=0.0, slab_width = zdim - 10, normal = (0.0,0.0,1.0)))

# delete selected modifier

pipeline.modifiers.append(DeleteSelectedModifier())


In [ ]:
data_last_frame = pipeline.compute(pipeline.source.num_frames - 1)
print(f"Particles in last frame: {data_last_frame.particles.count}") ## number of vacancies!! can be used as a quick check before running through the full calculation process

# last frame:
export_file(data_last_frame, "quartz_600k_atomic_1keV_28282_5ps_python.xyz", "xyz", frame = pipeline.source.num_frames - 1 , multiple_frames = False, columns =
  ["Particle Identifier", "Particle Type", "Position.X", "Position.Y", "Position.Z", "Timestep", "Coordination"])

# full run (for recombination analysis):
export_file(pipeline, "quartz_600k_atomic_1keV_28282_5ps_numvacancies_python.xyz", "xyz", multiple_frames = True, columns =
  ["Particle Identifier", "Particle Type", "Timestep"])